데이터 불러오기 및 데이터프레임 확인

In [3]:
import pandas as pd

# 파일 경로에 맞게 조정 
population_df = pd.read_csv(r"C:\Users\wanje\OneDrive\바탕 화면\a. 행정안전부_지역별(행정동) 성별 연령별 주민등록 인구수_20250331.csv", encoding='cp949')
school_df = pd.read_csv(r"C:\Users\wanje\OneDrive\바탕 화면\학교기본정보_2025년3월31일기준.csv", encoding='cp949')
library_df = pd.read_csv(r"C:\Users\wanje\OneDrive\바탕 화면\전국 도서관 읍면동.csv", encoding='utf-8')
mart_df = pd.read_csv(r"C:\Users\wanje\OneDrive\바탕 화면\전국 읍면동 마트 개수.csv", encoding='cp949')

# 각 데이터프레임 크기 확인
(population_df.shape, school_df.shape, library_df.shape, mart_df.shape)


((3614, 230), (12563, 25), (1548, 34), (4110, 30))

인구 데이터: 3,614개 읍면동

학교 데이터: 12,563개 학교

도서관 데이터: 1,548개 도서관

마트 데이터: 4,110개 점포

 인구 데이터 정제 및 생활권 클러스터링 (2~3만명 단위)

In [5]:

# 필요한 열 추출 및 컬럼명 정리
population_df_clean = population_df[['시도명', '시군구명', '읍면동명', '계']].copy()
population_df_clean.columns = ['시도', '시군구', '읍면동', '인구수']

# 인구수 숫자형으로 변환
population_df_clean['인구수'] = pd.to_numeric(population_df_clean['인구수'], errors='coerce')

# 인구 기준 내림차순 정렬
population_df_clean = population_df_clean.sort_values(by='인구수', ascending=False).reset_index(drop=True)

# 생활권 클러스터링: 누적 인구수 기준으로 그룹핑
cluster_id = 1
누적 = 0
cluster_ids = []

for _, row in population_df_clean.iterrows():
    누적 += row['인구수']
    cluster_ids.append(cluster_id)
    if 20000 <= 누적 <= 30000 or 누적 > 30000:
        cluster_id += 1
        누적 = 0

# 클러스터 ID 열 추가
population_df_clean['생활권ID'] = cluster_ids

# 결과 일부 확인
population_df_clean.head()


,시도,시군구,읍면동,인구수,생활권ID
0,경상남도,양산시,물금읍,116558,1
1,경기도,화성시,봉담읍,107849,2
2,경기도,남양주시,다산1동,105555,3
3,경기도,남양주시,진접읍,95246,4
4,충청남도,아산시,배방읍,92786,5


마트 데이터 전처리 및 생활권 인구 데이터와 병합

In [7]:
# 소재지 전체주소에서 시도/시군구/읍면동 추출
mart_df['읍면동'] = mart_df['소재지전체주소'].str.split().str[2]
mart_df['시군구'] = mart_df['소재지전체주소'].str.split().str[1]
mart_df['시도'] = mart_df['소재지전체주소'].str.split().str[0]

# 마트 수를 읍면동 단위로 집계
mart_df_grouped = mart_df.groupby(['시도', '시군구', '읍면동']).size().reset_index(name='마트수')

# 생활권 인구 데이터와 병합
infra_with_mart = pd.merge(population_df_clean, mart_df_grouped, on=['시도', '시군구', '읍면동'], how='left')
infra_with_mart['마트수'] = infra_with_mart['마트수'].fillna(0).astype(int)

# 결과 일부 확인
infra_with_mart.head()


,시도,시군구,읍면동,인구수,생활권ID,마트수
0,경상남도,양산시,물금읍,116558,1,2
1,경기도,화성시,봉담읍,107849,2,5
2,경기도,남양주시,다산1동,105555,3,0
3,경기도,남양주시,진접읍,95246,4,2
4,충청남도,아산시,배방읍,92786,5,5


학교 데이터 전처리 및 생활권 데이터와 병합

In [9]:
# 도로명주소에서 시도, 시군구, 읍면동 추출
school_df['읍면동'] = school_df['도로명주소'].str.split().str[2]
school_df['시군구'] = school_df['도로명주소'].str.split().str[1]
school_df['시도'] = school_df['도로명주소'].str.split().str[0]

# 학교 수를 읍면동 단위로 집계
school_grouped = school_df.groupby(['시도', '시군구', '읍면동']).size().reset_index(name='학교수')

# 마트 병합 결과와 병합
infra_with_school = pd.merge(infra_with_mart, school_grouped, on=['시도', '시군구', '읍면동'], how='left')
infra_with_school['학교수'] = infra_with_school['학교수'].fillna(0).astype(int)

# 결과 일부 확인
infra_with_school.head()


,시도,시군구,읍면동,인구수,생활권ID,마트수,학교수
0,경상남도,양산시,물금읍,116558,1,2,19
1,경기도,화성시,봉담읍,107849,2,5,18
2,경기도,남양주시,다산1동,105555,3,0,0
3,경기도,남양주시,진접읍,95246,4,2,16
4,충청남도,아산시,배방읍,92786,5,5,18


도서관 좌표 정리 및 생활권 중심점 계산 준비

In [11]:
# 도서관 주소에서 시도, 시군구, 읍면동 추출
library_df['읍면동'] = library_df['LBRRY_ADDR'].str.split().str[2]
library_df['시군구'] = library_df['LBRRY_ADDR'].str.split().str[1]
library_df['시도'] = library_df['LBRRY_ADDR'].str.split().str[0]

# 읍면동별 평균 위도(LA), 경도(LO) 계산
library_mean_coords = library_df.groupby(['시도', '시군구', '읍면동'])[['LBRRY_LA', 'LBRRY_LO']].mean().reset_index()
library_mean_coords.columns = ['시도', '시군구', '읍면동', '위도', '경도']

# 기존 인프라 데이터와 병합 (학교/마트까지 포함된 상태)
infra_with_coords = pd.merge(infra_with_school, library_mean_coords, on=['시도', '시군구', '읍면동'], how='left')

# 결과 일부 확인
infra_with_coords.head()


,시도,시군구,읍면동,인구수,생활권ID,마트수,학교수,위도,경도
0,경상남도,양산시,물금읍,116558,1,2,19,35.336523,129.020218
1,경기도,화성시,봉담읍,107849,2,5,18,37.222650,126.946545
2,경기도,남양주시,다산1동,105555,3,0,0,NaN,NaN
3,경기도,남양주시,진접읍,95246,4,2,16,37.722975,127.193304
4,충청남도,아산시,배방읍,92786,5,5,18,36.773912,127.057794


생활권 중심점 계산 및 도서관 수 계산 (반경 3km 기준)

In [19]:
# 생활권 중심 좌표 평균 (NaN 포함 가능성 있음)
cluster_centers = infra_with_coords.groupby('생활권ID')[['위도', '경도']].mean().reset_index()

# 도서관 좌표 정리 및 라디안 변환
library_coords = library_df[['LBRRY_LA', 'LBRRY_LO']].dropna()
library_coords.columns = ['위도', '경도']
library_coords_rad = np.deg2rad(library_coords[['위도', '경도']].to_numpy())

# 중심 좌표에서 NaN 제거
coords_deg = cluster_centers[['위도', '경도']].to_numpy()
coords_deg = coords_deg[~np.isnan(coords_deg).any(axis=1)]
valid_center_coords_rad = np.deg2rad(coords_deg)

# BallTree로 거리 기반 탐색
tree = BallTree(library_coords_rad, metric='haversine')
radius_rad = 3 / 6371  # 반경 3km

# 도서관 수 계산
counts = tree.query_radius(valid_center_coords_rad, r=radius_rad, count_only=True)

# 도서관 수가 포함된 생활권 중심점 데이터프레임 생성
valid_centers = cluster_centers.dropna(subset=['위도', '경도']).iloc[:len(counts)].copy()
valid_centers['도서관수'] = counts

# 결과 확인
valid_centers.head()


,생활권ID,위도,경도,도서관수
0,1,35.336523,129.020218,3
1,2,37.222650,126.946545,3
3,4,37.722975,127.193304,3
4,5,36.773912,127.057794,2
5,6,37.131123,126.922481,2


In [ ]:
생활권별 학교수, 마트수 합산 및 인프라 등급 평가

In [21]:

# 학교수, 마트수 생활권ID 기준으로 합산
infra_summary = infra_with_coords.groupby('생활권ID')[['학교수', '마트수']].sum().reset_index()

# 도서관 수 포함된 중심 좌표와 병합
infra_full = pd.merge(valid_centers, infra_summary, on='생활권ID', how='left')

# 인프라 적정성 등급 함수 정의
def evaluate_infra(row):
    score = int(row['학교수'] >= 1) + int(row['마트수'] >= 2) + int(row['도서관수'] >= 1)
    return '충분' if score == 3 else '보통' if score == 2 else '부족'

# 등급 적용
infra_full['등급'] = infra_full.apply(evaluate_infra, axis=1)

# 결과 확인
infra_full.head()


,생활권ID,위도,경도,도서관수,학교수,마트수,등급
0,1,35.336523,129.020218,3,19,2,충분
1,2,37.222650,126.946545,3,18,5,충분
2,4,37.722975,127.193304,3,16,2,충분
3,5,36.773912,127.057794,2,18,5,충분
4,6,37.131123,126.922481,2,20,2,충분


In [ ]:
생활권별 읍면동 목록, 총 인구수, 시도/시군구 목록 정리

In [23]:
# 생활권ID별 읍면동 목록
eup_list = infra_with_coords.groupby('생활권ID')['읍면동'].apply(lambda x: '; '.join(x)).reset_index(name='생활권_읍면동_목록')

# 생활권ID별 총 인구수
pop_sum = infra_with_coords.groupby('생활권ID')['인구수'].sum().reset_index(name='생활권_총인구수')

# 생활권ID별 시도 목록
sido_list = infra_with_coords.groupby('생활권ID')['시도'].apply(lambda x: '; '.join(sorted(x.dropna().unique()))).reset_index(name='생활권_시도_목록')

# 생활권ID별 시군구 목록
sigungu_list = infra_with_coords.groupby('생활권ID')['시군구'].apply(lambda x: '; '.join(sorted(x.dropna().unique()))).reset_index(name='생활권_시군구_목록')

# 병합
infra_extended = (
    infra_full
    .merge(eup_list, on='생활권ID', how='left')
    .merge(pop_sum, on='생활권ID', how='left')
    .merge(sido_list, on='생활권ID', how='left')
    .merge(sigungu_list, on='생활권ID', how='left')
)

# 결과 확인
infra_extended.head()


,생활권ID,위도,경도,도서관수,학교수,마트수,등급,생활권_읍면동_목록,생활권_총인구수,생활권_시도_목록,생활권_시군구_목록
0,1,35.336523,129.020218,3,19,2,충분,물금읍,116558,경상남도,양산시
1,2,37.222650,126.946545,3,18,5,충분,봉담읍,107849,경기도,화성시
2,4,37.722975,127.193304,3,16,2,충분,진접읍,95246,경기도,남양주시
3,5,36.773912,127.057794,2,18,5,충분,배방읍,92786,충청남도,아산시
4,6,37.131123,126.922481,2,20,2,충분,향남읍,85318,경기도,화성시


생활권별 반지름 계산 (생활권 중심점과 각 읍면동 중심점 거리 중 최대값)

In [ ]:
from geopy.distance import geodesic

# 각 읍면동의 평균 위경도
eup_coords = infra_with_coords.groupby(['생활권ID', '읍면동'])[['위도', '경도']].mean().reset_index()

# 생활권 중심 좌표 테이블 준비
center_coords = infra_extended.set_index('생활권ID')[['위도', '경도']]

# 반지름 계산
radius_list = []
for cluster_id, group in eup_coords.groupby('생활권ID'):
    if cluster_id in center_coords.index:
        center = tuple(center_coords.loc[cluster_id])
        max_dist = 0
        for _, row in group.iterrows():
            if not pd.isna(row['위도']) and not pd.isna(row['경도']):
                dist = geodesic(center, (row['위도'], row['경도'])).km
                max_dist = max(max_dist, dist)
        radius_list.append({'생활권ID': cluster_id, '생활권_반지름_km': round(max_dist, 2)})

radius_df = pd.DataFrame(radius_list)

# 중심점 좌표 문자열 추가
infra_extended['생활권_중심점'] = infra_extended.apply(
    lambda row: f"({row['위도']:.6f}, {row['경도']:.6f})", axis=1
)

# 반지름 병합
final_result = pd.merge(infra_extended, radius_df, on='생활권ID', how='left')

# 결과 확인
final_result.head()


최종 결과 CSV 파일로 저장

In [31]:
# 반지름 데이터와 병합 완료된 상태에서 저장
output_path = "/mnt/data/생활권별_인프라_최종결과_.csv"

final_result.to_csv("생활권별_인프라_최종결과_완전버전.csv", index=False, encoding='utf-8-sig')
output_path


'/mnt/data/생활권별_인프라_최종결과_.csv'